In [1]:
def FindAllGroups(store_domain):
    # libraries
    import numpy as np
    import pandas as pd
    import json
    import operator
    import requests
    import matplotlib.pyplot as plt
    from PIL import Image
    import requests
    from io import BytesIO

    import requests
    from io import BytesIO
    from PIL import Image

    from sklearn.cluster import DBSCAN

    from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
    from sklearn.metrics import homogeneity_score, completeness_score

    from sklearn.feature_extraction.text import TfidfVectorizer

    import cv2
    from skimage.feature import hog

    import numpy as np
    import cv2
    from tensorflow.keras.applications.vgg16 import VGG16
    from tensorflow.keras.applications.vgg16 import preprocess_input as preprocess_vgg16
    from tensorflow.keras.applications.resnet50 import ResNet50
    from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet50
    from sklearn.cluster import KMeans
    
    model = VGG16(weights='imagenet', include_top=False)
    
    # function to sort dictionary
    def frequency_dict_sorted(input_list):
        freq_dict = {}
        for item in input_list:
            if item in freq_dict:
                freq_dict[item] += 1
            else:
                freq_dict[item] = 1

        freq_dict_sorted = dict(sorted(freq_dict.items(), key=operator.itemgetter(1),reverse=True))
        return freq_dict_sorted
    
    
    
    def load_store_domain_data(store_domain):
        products_url = f"{store_domain}/collections/all/products.json"
        all_products = []
        page_number = 1

        # fetch all products from the store
        while True:
            page_url = f"{products_url}?page={page_number}"
            response = requests.get(page_url)
            if response.status_code != 200:
                break
            products_data = response.json()
            if not products_data:
                break
            all_products.extend(products_data['products'])
            page_number += 1
        return all_products
    
    # storing the data in a dataframe
    def store_dataframe(all_products):
        data = pd.DataFrame()
        product_title_list = []
        product_handle_list = []
        product_vendor_list = []
        product_type_list = []
        product_tags_list = []

        for i in range(len(all_products)):
            product_title_list.append(all_products[i]['title'].lower())
            product_handle_list.append(all_products[i]['handle'].lower())
            product_vendor_list.append(all_products[i]['vendor'].lower())
            product_type_list.append(all_products[i]['product_type'].lower())
            product_tags_list.append(all_products[i]['tags'])

        data['title'] = product_title_list
        data['handle'] = product_handle_list
        data['vendor'] = product_vendor_list
        data['product_type'] = product_type_list
        data['tags'] = product_tags_list
        return data
    
    
    
    def load_images(a,b,product_type,color=True):
        out_list = []
        idx_list = []
        skipped_list = []
        faulty_list = []
        for i in range(a,b):
            try:
                if (all_products[i]['product_type'].lower()==product_type) or (product_type=='ALL'):
                    url = all_products[i]['images'][0]['src']
                    response = requests.get(url)
                    img_buffer = BytesIO(response.content)
                    img = Image.open(img_buffer)
                    img = img.resize((224, 224), Image.BILINEAR)
                    img_array = np.array(img)
                    gray_img = cv2.cvtColor(img_array, cv2.COLOR_RGB2GRAY)
                    if color==True:
                        out_list.append(img)
                        idx_list.append(i)
                    else:

                        out_list.append(gray_img)
                        idx_list.append(i)
                else:
                    skipped_list.append(i)
            except:
                faulty_list.append(i)
    #         out = np.array(out_list)
        return out_list,idx_list,skipped_list,faulty_list
    
    
    def canny_edge(img):
        blur = cv2.GaussianBlur(img, (5,5), 0)
        edges = cv2.Canny(blur, 50, 200)
        return edges
    def sobel_edge(img):
        blur = cv2.GaussianBlur(img, (5,5), 0)
        sobelx = cv2.Sobel(blur, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(blur, cv2.CV_64F, 0, 1, ksize=3)
        edges = cv2.bitwise_or(sobelx, sobely)
        return edges
    
    
    # Define function to extract VGG16 features from an image
    def extract_features(x):
    #     img = cv2.imread(img_path)
    #     img = cv2.resize(img, (224, 224))
    #     img = np.expand_dims(img, axis=0)
    #     x = np.array(img)
        x = np.expand_dims(x, axis=2)
        x = np.repeat(x, 3, axis=2)
        x = np.expand_dims(x, axis=0)
    #     print(x.shape)
        x = preprocess_vgg16(x)
        features = model.predict(x)
    #     print(features.shape)
        features = features.flatten()
        return features
    
    
    
    
    
    
    all_products = load_store_domain_data(store_domain)
    print(f"all products = {len(all_products)}")
    data = store_dataframe(all_products)
    print(f"data shape = {data.shape}")
    test,idx_list,skipped_list,faulty_list = load_images(0,len(all_products),"ALL",False)
    data.drop(index=faulty_list,inplace=True)
    print(f"data shape = {data.shape}")

    features_list = []
    for idx, img in enumerate(test):
        try:
    #         print(img.shape)
            img = canny_edge(img)
            features = extract_features(img)
            features_list.append(features)
        except:
            print(idx)
    features_array = np.array(features_list)
    print(f"feature array shape = {features_array.shape}")
    # Perform clustering using K-means algorithm
    n_clusters = len(set(data['product_type']))
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(features_array)

    # Print cluster labels
    print(kmeans.labels_)
    silhouette_score(features_array,kmeans.labels_)
    clusters_list = list(frequency_dict_sorted(kmeans.labels_).keys()) 
    data['labels'] = kmeans.labels_
    output_list = []

    # preparation for the json output
    for i in clusters_list:
        product_varients = []
        temp = data[data['labels'] == i]
        for i in range(len(temp)):
            product_link = f"{store_domain}/products/{temp['handle'].iloc[i]}"
            product_varients.append(product_link)
        product_varients_dict = {}
        product_varients_dict["product variations"] = product_varients
        output_list.append(product_varients_dict)
        output = json.dumps(output_list)

    return output

In [2]:
store_domain = "https://www.boysnextdoor-apparel.co"
# store_domain = "https://www.woolsboutiqueuomo.com"
# store_domain = "https://sartale2022.myshopify.com"
# store_domain = "https://berkehome.pl"
# store_domain = "https://glamaroustitijewels.com"
# store_domain = "https://lampsdepot.com"
# store_domain = "https://kitchenoasis.com"

In [3]:
FindAllGroups(store_domain)

58889256/58889256 [==============================] - 0s 0us/step
all products = 670
data shape = (670, 5)
data shape = (661, 5)
1/1 [==============================] - 0s 280ms/step
feature array shape = (661, 25088)
[17  3  0  0 12 20  9  1 18 17 10 10 17  1  1 21 21 20 19 19 20  6  9  9
 20  2 12 12 12 17 10  6  6 24 20 24 14  6  6  7 10  9 23 14  9  2  9 13
 13 13 13 13 13  8  8  8 25 17 10 10  6  6  6 25 25 18 20 21 10  7  7 24
 10 24  8 14  6  6 23 25 18  8  8 10 10 10 10  9  9  6  6  8 23 23 23 24
  8  8  8 24  8 14 20 20 20 18  9  6  5 20  4  9  4  4 17 20  9  9 24 14
  9  9 20 18  9  8 19  8 20  6  5  8  8 21  1  1 19 20 18 20 20 24 20 24
 24  8  8  6  6  9  1  9  9  1  8  8  8 14 17 17  9  2  2  9  1  4 18  6
  4  1 10 10 17  6  6 24  6 24  6  1 18 23  6 24  9  1  4  4  4 14 20 18
 14 16  8  7  7  7  7  7  7  7  7 20 10  7 10  7 17 24 14 20  7  8  8  0
  3  3  3 10 17 22 17  7 10  8  8 18 18 20 14 19 20 25  9 20 25 18 18 18
  1 24 24 10 10 10  7  7  7  7  7 10 10  7 10 10 14 19

'[{"product variations": ["https://www.boysnextdoor-apparel.co/products/beams-two-pocket-sweater-white", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-coach-shirt-jacket-white", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-fishtail-ls-tshirt-white", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-hunting-jacket-beige", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-mock-pocket-pullover-charcoal", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-oversize-stripes-tee", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-pullover-shirt-jacket-white", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-reversible-vest-black", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-round-collar-shirt-white-1", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-texture-vest-beige", "https://www.boysnextdoor-apparel.co/products/boysnextdoor-texture-vest-navy", "https://www.boysnextdoor-apparel.co/products/boy